# Loading Credentials

In [1]:
import pickle
pickle_in = open("credentials.pickle","rb")
creds = pickle.load(pickle_in)

By using the pickle file, you can create 'raw dumps' of variables or any other form of content you would use in your code. Here, my Spotify API Client ID and Client Secret is stored in the credentials.pickle file, so one doesn't need to keep that code in the notebook.

In [5]:
import spotify
spt = spotify.Client(client_id=creds['ClientID'],
                    client_secret=creds['ClientSecrets'])